In [38]:
import numpy as np
import netCDF4 as nc
import os,sys

transcom_filename = '/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/transcom_regions_60min.nc'
transcom_fh = nc.Dataset(transcom_filename,mode='r')
transcom = transcom_fh.variables['transcom_regions'][:]
print np.shape(transcom)

CO2_filename = '/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/fossil_gcb_data/fossil_gcb_monthly_2000_2014.nc'
#CO2_filename = '/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/fossil_gcb_data/test.nc'
CO2_fh = nc.Dataset(CO2_filename,mode='r')

CO2 = CO2_fh.variables['fossilfuel_burning_carbon_flux'][:] # months,lats,lon
times = CO2_fh.variables['time'][:]
CO2_fh.close()


CO2_mean = np.zeros([len(times),180,360])
CO2_sum = np.zeros([len(times),180,360])
for timestep in range(len(times)):
    CO2_step = CO2[timestep,:,:]
    for i in range(23):
        i+=1
        #print(i)
        mask = np.where(transcom==i,True,False)
        CO2_mean[timestep,:,:][mask]=np.mean(CO2_step[mask])
        CO2_sum[timestep,:,:][mask]=np.sum(CO2_step[mask])

# Total Emissions - Sum
os.remove('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/fossil_gcb_data/transcom_gcb_monthly_sum_2000_2014.nc')
dataset = nc.Dataset('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/fossil_gcb_data/transcom_gcb_monthly_sum_2000_2014.nc','w',format='NETCDF4_CLASSIC')
lat = dataset.createDimension('lat', 180)
lon = dataset.createDimension('lon', 360)
time = dataset.createDimension('time', None)
for dimname in dataset.dimensions.keys():
    dim = dataset.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()
# Create coordinate variables for 4-dimensions
time_out = dataset.createVariable('time', np.float64, ('time',))
lat_out = dataset.createVariable('latitude', np.float32, ('lat',))
lon_out = dataset.createVariable('longitude', np.float32, ('lon',)) 
# Create the actual 4-d variable
emissions_sum = dataset.createVariable('emissions_sum', np.float32,('time','lat','lon'))

for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape
    
# Fill the file
lat_out[:] = np.arange(-89.5,90.5,1)
lon_out[:] = np.arange(-179.5,180.5,1)
time_out[:]= times
emissions_sum[:,:,:]=CO2_sum
dataset.close()

# Average Emissions - Mean
os.remove('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/fossil_gcb_data/transcom_gcb_monthly_mean_2000_2014.nc')
dataset = nc.Dataset('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/fossil_gcb_data/transcom_gcb_monthly_mean_2000_2014.nc','w',format='NETCDF4_CLASSIC')
lat = dataset.createDimension('lat', 180)
lon = dataset.createDimension('lon', 360)
time = dataset.createDimension('time', None)
for dimname in dataset.dimensions.keys():
    dim = dataset.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()
# Create coordinate variables for 4-dimensions
time_out = dataset.createVariable('time', np.float64, ('time',))
lat_out = dataset.createVariable('latitude', np.float32, ('lat',))
lon_out = dataset.createVariable('longitude', np.float32, ('lon',)) 
# Create the actual 4-d variable
emissions_mean = dataset.createVariable('emissions_mean', np.float32,('time','lat','lon'))

for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape
    
# Fill the file
lat_out[:] = np.arange(-89.5,90.5,1)
lon_out[:] = np.arange(-179.5,180.5,1)
time_out[:]= times
emissions_mean[:,:,:]=CO2_mean
dataset.close()


####################
####    CMIP    ####
####################

CMIP_filename = '/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/2D-SECTORS/CO2-em-anthro_input4MIPs_emissions_CMIP_CEDS-2017-05-18_gn_200001-201412_60min.nc'
CMIP_fh = nc.Dataset(CMIP_filename,mode='r')

CMIP = CMIP_fh.variables['CO2_em_anthro'][:]
lons = CMIP_fh.variables['lon'][:]
lats = CMIP_fh.variables['lat'][:]
times = CMIP_fh.variables['time'][:]
sectors = CMIP_fh.variables['sector'][:]
CMIP_fh.close()

CMIP_mean = np.zeros([len(times),len(sectors),len(lats),len(lons)])
CMIP_sum = np.zeros([len(times),len(sectors),len(lats),len(lons)])
CMIP_mean_tot = np.zeros([len(times),len(lats),len(lons)])
CMIP_sum_tot = np.zeros([len(times),len(lats),len(lons)])
for timestep in range(len(times)):
    for sectorstep in range(len(sectors)):
        CMIP_step = CMIP[timestep,sectorstep,:,:]
        for i in range(23):
            i+=1
            #print(i)
            mask = np.where(transcom==i,True,False)
            CMIP_mean[timestep,sectorstep,:,:][mask]=np.mean(CMIP_step[mask])
            CMIP_sum[timestep,sectorstep,:,:][mask]=np.sum(CMIP_step[mask])
            CMIP_mean_tot[timestep,:,:][mask]=CMIP_mean_tot[timestep,:,:][mask]+np.mean(CMIP_step[mask])
            CMIP_sum_tot[timestep,:,:][mask]=CMIP_sum_tot[timestep,:,:][mask]+np.sum(CMIP_step[mask])
            
# Total Emissions - Sum
os.remove('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/CMIP-Transcom/CMIP_transcom_monthly_sum_2000_2014.nc')
dataset = nc.Dataset('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/CMIP-Transcom/CMIP_transcom_monthly_sum_2000_2014.nc','w',format='NETCDF4_CLASSIC')
sector = dataset.createDimension('sector', 8)
lat = dataset.createDimension('lat', 180)
lon = dataset.createDimension('lon', 360)
time = dataset.createDimension('time', None)
for dimname in dataset.dimensions.keys():
    dim = dataset.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()
# Create coordinate variables for 4-dimensions
time_out = dataset.createVariable('time', np.float64, ('time',))
sector_out = dataset.createVariable('sector', np.int32, ('sector',))
lat_out = dataset.createVariable('latitude', np.float32, ('lat',))
lon_out = dataset.createVariable('longitude', np.float32, ('lon',)) 
# Create the actual 4-d variable
emissions_sum = dataset.createVariable('emissions_sum_CMIP', np.float32,('time','sector','lat','lon'))

for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape
    
# Fill the file
lat_out[:] = lats
lon_out[:] = lons
time_out[:]= times
sector_out[:]=sectors
emissions_sum[:,:,:,:]=CMIP_sum
dataset.close()

# Average Emissions - Mean
os.remove('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/CMIP-Transcom/CMIP_transcom_monthly_mean_2000_2014.nc')
dataset = nc.Dataset('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/CMIP-Transcom/CMIP_transcom_monthly_mean_2000_2014.nc','w',format='NETCDF4_CLASSIC')
sector = dataset.createDimension('sector', 8)
lat = dataset.createDimension('lat', 180)
lon = dataset.createDimension('lon', 360)
time = dataset.createDimension('time', None)
for dimname in dataset.dimensions.keys():
    dim = dataset.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()
# Create coordinate variables for 4-dimensions
time_out = dataset.createVariable('time', np.float64, ('time',))
sector_out = dataset.createVariable('sector', np.int32, ('sector',))
lat_out = dataset.createVariable('latitude', np.float32, ('lat',))
lon_out = dataset.createVariable('longitude', np.float32, ('lon',)) 
# Create the actual 4-d variable
emissions_mean = dataset.createVariable('emissions_mean_CMIP', np.float32,('time','sector','lat','lon'))

for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape
    
# Fill the file
lat_out[:] = lats
lon_out[:] = lons
time_out[:]= times
sector_out[:]=sectors
emissions_mean[:,:,:,:]=CMIP_mean
dataset.close()

# Total Emissions - Sum - No sectors
#os.remove('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/CMIP-Transcom/CMIP_transcom_monthly_sum_sectorsummed_2000_2014.nc')
dataset = nc.Dataset('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/CMIP-Transcom/CMIP_transcom_monthly_sum_sectorsummed_2000_2014.nc','w',format='NETCDF4_CLASSIC')
lat = dataset.createDimension('lat', 180)
lon = dataset.createDimension('lon', 360)
time = dataset.createDimension('time', None)
for dimname in dataset.dimensions.keys():
    dim = dataset.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()
# Create coordinate variables for 4-dimensions
time_out = dataset.createVariable('time', np.float64, ('time',))
lat_out = dataset.createVariable('latitude', np.float32, ('lat',))
lon_out = dataset.createVariable('longitude', np.float32, ('lon',)) 
# Create the actual 4-d variable
emissions_sum = dataset.createVariable('emissions_sum_CMIP', np.float32,('time','lat','lon'))

for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape
    
# Fill the file
lat_out[:] = np.arange(-89.5,90.5,1)
lon_out[:] = np.arange(-179.5,180.5,1)
time_out[:]= times
emissions_sum[:,:,:]=CMIP_sum_tot
dataset.close()

# Average Emissions - Mean - No sectors
#os.remove('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/CMIP-Transcom/CMIP_transcom_monthly_mean_sectorsummed_2000_2014.nc')
dataset = nc.Dataset('/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/CO2-emissiebestanden-CMIP6/CMIP-Transcom/CMIP_transcom_monthly_mean_sectorsummed_2000_2014.nc','w',format='NETCDF4_CLASSIC')
lat = dataset.createDimension('lat', 180)
lon = dataset.createDimension('lon', 360)
time = dataset.createDimension('time', None)
for dimname in dataset.dimensions.keys():
    dim = dataset.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()
# Create coordinate variables for 4-dimensions
time_out = dataset.createVariable('time', np.float64, ('time',))
lat_out = dataset.createVariable('latitude', np.float32, ('lat',))
lon_out = dataset.createVariable('longitude', np.float32, ('lon',)) 
# Create the actual 4-d variable
emissions_mean = dataset.createVariable('emissions_mean_CMIP', np.float32,('time','lat','lon'))

for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape
    
# Fill the file
lat_out[:] = np.arange(-89.5,90.5,1)
lon_out[:] = np.arange(-179.5,180.5,1)
time_out[:]= times
emissions_mean[:,:,:]=CMIP_mean_tot
dataset.close()

(180, 360)
lat 180 False
lon 360 False
time 0 True
time float64 (u'time',) (0,)
latitude float32 (u'lat',) (180,)
longitude float32 (u'lon',) (360,)
emissions_sum float32 (u'time', u'lat', u'lon') (0, 180, 360)
lat 180 False
lon 360 False
time 0 True
time float64 (u'time',) (0,)
latitude float32 (u'lat',) (180,)
longitude float32 (u'lon',) (360,)
emissions_mean float32 (u'time', u'lat', u'lon') (0, 180, 360)
sector 8 False
lat 180 False
lon 360 False
time 0 True
time float64 (u'time',) (0,)
sector int32 (u'sector',) (8,)
latitude float32 (u'lat',) (180,)
longitude float32 (u'lon',) (360,)
emissions_sum float32 (u'time', u'sector', u'lat', u'lon') (0, 8, 180, 360)
sector 8 False
lat 180 False
lon 360 False
time 0 True
time float64 (u'time',) (0,)
sector int32 (u'sector',) (8,)
latitude float32 (u'lat',) (180,)
longitude float32 (u'lon',) (360,)
emissions_mean float32 (u'time', u'sector', u'lat', u'lon') (0, 8, 180, 360)
lat 180 False
lon 360 False
time 0 True
time float64 (u'time',) (0,

In [30]:
%reset
import numpy as np
import netCDF4 as nc
import os

transcom_filename = '/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/transcom_regions_30min.nc'
transcom_fh = nc.Dataset(transcom_filename,mode='r')
transcom = transcom_fh.variables['transcom_regions'][:]
print np.shape(transcom)

CO2_filename = '/nobackup/users/killaars/PhD_data/CO2_emission_comparison_data/fossil_gcb_data/fossil_gcb_monthly_2000_2014.nc'
CO2_fh = nc.Dataset(CO2_filename,mode='r')

CO2 = CO2_fh.variables['CO2_em_anthro'][:]
lons = CO2_fh.variables['lon'][:]
lats = CO2_fh.variables['lat'][:]
times = CO2_fh.variables['time'][:]
sectors = CO2_fh.variables['sector'][:]
CO2_fh.close()

CO2_mean = np.zeros([len(times),len(sectors),len(lats),len(lons)])
CO2_sum = np.zeros([len(times),len(sectors),len(lats),len(lons)])
miniarray = np.arange(2)
for timestep in range(len(times)):
    for sectorstep in range(len(sectors)):
        CO2_step = CO2[timestep,sectorstep,:,:]
        for i in range(23):
            i+=1
            #print(i)
            mask = np.where(transcom==i,True,False)
            CO2_mean[timestep,sectorstep,:,:][mask]=np.mean(CO2_step[mask])
            CO2_sum[timestep,sectorstep,:,:][mask]=np.sum(CO2_step[mask])
            
# Total Emissions - Sum
#os.remove('/nobackup/users/killaars/PhD_data/CO2-emissiebestanden-CMIP6/Emissions_sum_1750-1799.nc')
dataset = nc.Dataset('/nobackup/users/killaars/PhD_data/CO2-emissiebestanden-CMIP6/Emissions_sum_1800-1849.nc','w',format='NETCDF4_CLASSIC')
sector = dataset.createDimension('sector', 8)
lat = dataset.createDimension('lat', 360)
lon = dataset.createDimension('lon', 720)
time = dataset.createDimension('time', None)
for dimname in dataset.dimensions.keys():
    dim = dataset.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()
# Create coordinate variables for 4-dimensions
time_out = dataset.createVariable('time', np.float64, ('time',))
sector_out = dataset.createVariable('sector', np.int32, ('sector',))
lat_out = dataset.createVariable('latitude', np.float32, ('lat',))
lon_out = dataset.createVariable('longitude', np.float32, ('lon',)) 
# Create the actual 4-d variable
emissions_sum = dataset.createVariable('emissions_sum', np.float32,('time','sector','lat','lon'))

for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape
    
# Fill the file
lat_out[:] = lats
lon_out[:] = lons
time_out[:]= times
sector_out[:]=sectors
emissions_sum[:,:,:,:]=CO2_sum
dataset.close()

# Average Emissions - Mean
#os.remove('/nobackup/users/killaars/PhD_data/CO2-emissiebestanden-CMIP6/Emissions_mean_1750-1799.nc')
dataset = nc.Dataset('/nobackup/users/killaars/PhD_data/CO2-emissiebestanden-CMIP6/Emissions_mean_1800-1849.nc','w',format='NETCDF4_CLASSIC')
sector = dataset.createDimension('sector', 8)
lat = dataset.createDimension('lat', 360)
lon = dataset.createDimension('lon', 720)
time = dataset.createDimension('time', None)
for dimname in dataset.dimensions.keys():
    dim = dataset.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()
# Create coordinate variables for 4-dimensions
time_out = dataset.createVariable('time', np.float64, ('time',))
sector_out = dataset.createVariable('sector', np.int32, ('sector',))
lat_out = dataset.createVariable('latitude', np.float32, ('lat',))
lon_out = dataset.createVariable('longitude', np.float32, ('lon',)) 
# Create the actual 4-d variable
emissions_mean = dataset.createVariable('emissions_mean', np.float32,('time','sector','lat','lon'))

for varname in dataset.variables.keys():
    var = dataset.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape
    
# Fill the file
lat_out[:] = lats
lon_out[:] = lons
time_out[:]= times
sector_out[:]=sectors
emissions_mean[:,:,:,:]=CO2_mean
dataset.close()


(720,)
(360,)
(265,)
(8,)


sector 8 False
lat 360 False
lon 720 False
time 0 True
time float64 (u'time',) (0,)
sector int32 (u'sector',) (8,)
latitude float32 (u'lat',) (360,)
longitude float32 (u'lon',) (720,)
emissions_sum float32 (u'time', u'sector', u'lat', u'lon') (0, 8, 360, 720)
sector 8 False
lat 360 False
lon 720 False
time 0 True
time float64 (u'time',) (0,)
sector int32 (u'sector',) (8,)
latitude float32 (u'lat',) (360,)
longitude float32 (u'lon',) (720,)
emissions_mean float32 (u'time', u'sector', u'lat', u'lon') (0, 8, 360, 720)


In [105]:
test=np.arange(9)+1
test = np.reshape(test,(3,3))
testmask = np.where(test>6,True,False)
print(test)
print(testmask)
test[testmask] = np.sum(test[testmask])
test

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[False False False]
 [False False False]
 [ True  True  True]]


array([[ 1,  2,  3],
       [ 4,  5,  6],
       [24, 24, 24]])

In [24]:
CO2_mean = np.zeros([2,2,5,5])
CO2_sum = np.zeros([2,2,5,5])
testarray = np.ones([2,2,5,5])
print(testarray)
testmask = np.reshape((np.arange(25)+1),(5,5))
print(testmask)

for timestep in range(2):
    for sectorstep in range(2):
        test=testarray[timestep,sectorstep,:,:]
        print(test)
        mask = np.where(testmask>20,True,False)
        CO2_mean[timestep,sectorstep,:,:][mask] = np.mean(test[mask])
        CO2_sum[timestep,sectorstep,:,:][mask] = np.sum(test[mask])
        
print(CO2_mean)
print(CO2_sum)

[[[[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]

  [[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]]


 [[[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]

  [[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]]]
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]
[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]
[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]
[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1